In [8]:
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker


geolocator = Nominatim(user_agent="kaggle_learn")
location = geolocator.geocode("Pyramid of Khufu")
print(location.point)
print(location.address)

point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

29 58m 44.976s N, 31 8m 3.17625s E
هرم خوفو, شارع ابو الهول السياحي, نزلة البطران, الجيزة, 12556, مصر
Latitude: 29.97916
Longitude: 31.134215625236113


In [9]:
universities = pd.read_csv("top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [10]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

universities[['Latitude', 'Longitude']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(universities["Latitude"])) / len(universities)) * 100))

# Drop universities that were not successfully geocoded
universities = universities.loc[~np.isnan(universities["Latitude"])]
universities = gpd.GeoDataFrame(
    universities, geometry=gpd.points_from_xy(universities.Longitude, universities.Latitude))
universities.crs = ('epsg:4326')
universities.head()

94.0% of addresses were geocoded!


,Name,Latitude,Longitude,geometry
0,University of Oxford,51.752546,-1.214330,POINT (-1.21433 51.75255)
1,University of Cambridge,52.200623,0.110474,POINT (0.11047 52.20062)
2,Imperial College London,51.498959,-0.175641,POINT (-0.17564 51.49896)
3,ETH Zurich,47.376504,8.547321,POINT (8.54732 47.37650)
4,UCL,51.521764,-0.135120,POINT (-0.13512 51.52176)


In [11]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to the map
for idx, row in universities.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
m